# Tool choice

Tool use supports a parameter called `tool_choice` that allows you to specify how you want Claude to call tools. In this notebook, we'll take a look at how it works and when to use it. Before going any further, make sure you are comfortable with the basics of tool use with Claude.

When working with the `tool_choice` parameter, we have three possible options: 

* `auto` allows Claude to decide whether to call any provided tools or not
* `tool` allows us to force Claude to always use a particular tool
* `any` tells Claude that it must use one of the provided tools, but doesn't force a particular tool

Let's take a look at each option in detail. We'll start by importing the Anthropic SDK:

In [1]:
import os
import boto3

# "anthropic.claude-3-haiku-20240307-v1:0"  "anthropic.claude-3-sonnet-20240229-v1:0"  "anthropic.claude-3-opus-20240229-v1:0"
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

region_name= os.getenv("AWS_REGION", default="us-west-2")
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region_name)

## Auto

Setting `tool_choice` to `auto` allows the model to automatically decide whether to use tools or not.  This is the default behavior when working with tools. 

To demonstrate this, we're going to provide Claude with a fake web search tool. We will ask Claude questions, some of which would require calling the web search tool and other which Claude should be able to answer on its own.

Let's start by defining a tool called `web_search`.  Please note, to keep this demo simple, we're not actually searching the web here:

In [2]:
def web_search(topic):
    print(f"pretending to search the web for {topic}")

web_search_tool = {
    "name": "web_search",
    "description": "A tool to retrieve up to date information on a given topic by searching the web",
    "inputSchema": {
        "json": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to search the web for"
                }
            },
            "required": ["topic"]
        }
    }
}  


Next, we write a function that accepts a user_query and passes it along to Claude, along with the `web_search_tool`. 

We also set `tool_choice` to `auto`:

```py
tool_choice={"type": "auto"}
```

Here's the complete function:

In [3]:
from datetime import date

def chat_with_web_search(user_query):
    messages = [{"role": "user", "content": [{"text": user_query}]}]

    system_prompt=f"""
    Answer as many questions as you can using your existing knowledge.  
    Only search the web for queries that you can not confidently answer.
    Today's date is {date.today().strftime("%B %d %Y")}
    If you think a user's question involves something in the future that hasn't happened yet, use the search tool.
    """

    converse_response = bedrock_client.converse(
        system=[{"text" : system_prompt}],
        modelId=MODEL_ID,
        messages = messages,
        inferenceConfig = {"maxTokens": 4096},
        toolConfig = {
             'tools': [
                {
                    'toolSpec': web_search_tool
                }
            ]
        }
    )
    
    last_content_block = converse_response['output']['message']['content'][-1]
    if last_content_block.get("text"):
        print("Claude did NOT call a tool")
        print(f"Assistant: {last_content_block['text']}")
    elif last_content_block.get("toolUse"):
        print("Claude wants to use a tool")
        print(last_content_block)

Let's start with a question Claude should be able to answer without using the tool:

In [4]:
chat_with_web_search("What color is the sky?")

Claude did NOT call a tool
Assistant: The sky appears blue during the daytime. This is because of the way sunlight interacts with the gases in the Earth's atmosphere. The blue color of the sky is caused by the scattering of sunlight by the gas molecules in the atmosphere.


When we ask "What color is the sky?", Claude does not use the tool.  Let's try asking something that Claude should use the web search tool to answer:

In [5]:
chat_with_web_search("Who won the 2024 Miami Grand Prix?")

Claude wants to use a tool
{'toolUse': {'toolUseId': 'tooluse_3Re2lk8AR1q9dZkBWiFq3Q', 'name': 'web_search', 'input': {'topic': '2024 miami grand prix winner'}}}


When we ask "Who won the 2024 Miami Grand Prix?", Claude uses the web search tool! 

Let's try a few more examples:

In [6]:
# Claude should NOT need to use the tool for this:
chat_with_web_search("Who won the superbowl in 2022?")

Claude did NOT call a tool
Assistant: The Los Angeles Rams won Super Bowl LVI in 2022, defeating the Cincinnati Bengals by a score of 23-20. The game was played on February 13, 2022 at SoFi Stadium in Inglewood, California.


In [7]:
# Claude SHOULD use the tool for this:
chat_with_web_search("Who won the superbowl in 2024?")

Claude wants to use a tool
{'toolUse': {'toolUseId': 'tooluse_-9JRnkcBTxy5fGyzhKPmlQ', 'name': 'web_search', 'input': {'topic': '2024 super bowl winner'}}}


### Your Prompt Matters!

When working with `tool_choice` of `auto`, it's important that you spend time to write a detailed prompt.  Often, Claude can be over-eager to call tools.  Writing a detailed prompt helps Claude determine when to call a tool and when not to.  In the above example, we included specific instructions in the system prompt: 


```py
 system_prompt=f"""
    Answer as many questions as you can using your existing knowledge.  
    Only search the web for queries that you can not confidently answer.
    Today's date is {date.today().strftime("%B %d %Y")}
    If you think a user's question involves something in the future that hasn't happened yet, use the search tool.
"""
```



## Forcing a specific tool

We can force Claude to use a particular tool using `tool_choice`.  In the example below, we've defined two simple tools: 
* `print_sentiment_scores` - a tool that "tricks" Claude into generating well-structured JSON output containing sentiment analysis data.  For more info on this approach, see [Extracting Structured JSON using Claude and Tool Use](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/extracting_structured_json.ipynb)
* `calculator` - a very simple calculator tool that takes two numbers and adds them together 


In [8]:
tools = [
    {
        "toolSpec": {
            "name": "print_sentiment_scores",
            "description": "Prints the sentiment scores of a given tweet or piece of text.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "positive_score": {
                            "type": "number",
                            "description": "The positive sentiment score, ranging from 0.0 to 1.0."
                        },
                        "negative_score": {
                            "type": "number",
                            "description": "The negative sentiment score, ranging from 0.0 to 1.0."
                        },
                        "neutral_score": {
                            "type": "number",
                            "description": "The neutral sentiment score, ranging from 0.0 to 1.0."
                        }
                    },
                    "required": ["positive_score", "negative_score", "neutral_score"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "calculator",
            "description": "Adds two numbers",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "num1": {
                            "type": "number",
                            "description": "first number to add"
                        },
                        "num2": {
                            "type": "number",
                            "description": "second number to add"
                        }
                    },
                    "required": ["num1", "num2"]
                }
            }
        }
    }
]

Our goal is to write a function called `analyze_tweet_sentiment` that takes a tweet and prints a basic sentiment analysis of that tweet.  Eventually we will "force" Claude to use our sentiment analysis tool, but we'll start by showing what happens when we **do not** force the tool use. 

In this first "bad" version of the `analyze_tweet_sentiment` function, we provide Claude with both tools. For the sake of comparison, we'll start by setting tool_choice to "auto":

```py
tool_choice={"type": "auto"}
```

Please note that we are deliberately not providing Claude with a well-written prompt, to make it easier to see the impact of forcing the use of a particular tool.

In [9]:
def analyze_tweet_sentiment(query):
    converse_response = bedrock_client.converse(
        modelId=MODEL_ID,
        messages = [{
            "role": "user",
            "content": [{"text": query}]
        }],
        inferenceConfig = {"maxTokens": 4096},
        toolConfig = {
            'tools': [
                {
                    'toolSpec': web_search_tool
                }
            ],
            "toolChoice": { 'auto': {} },
        }
    )
    print(converse_response)

Let's see what happens when we call the function with the tweet "Holy cow, I just made the most incredible meal!"

In [10]:
analyze_tweet_sentiment("Holy cow, I just made the most incredible meal!")

{'ResponseMetadata': {'RequestId': 'b3a550b9-db96-4c6c-832d-1e07a0a91ab4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 11 Jun 2024 11:42:20 GMT', 'content-type': 'application/json', 'content-length': '405', 'connection': 'keep-alive', 'x-amzn-requestid': 'b3a550b9-db96-4c6c-832d-1e07a0a91ab4'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': "That's great to hear! I'd love to hear more about the amazing meal you prepared. What did you make? I'm always eager to learn about new recipes and culinary creations from humans. Please tell me all the delicious details!"}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 253, 'outputTokens': 53, 'totalTokens': 306}, 'metrics': {'latencyMs': 3652}}


Claude does not call our sentiment analysis tool:
> "That's great to hear! I don't actually have the capability to assess sentiment from text, but it sounds like you're really excited and proud of the incredible meal you made

Next, let's imagine someone tweets this: "I love my cats! I had four and just adopted 2 more! Guess how many I have now?"

In [14]:
analyze_tweet_sentiment("I love my cats! I had four and just adopted 2 more! Figure how many I have now?")

{'ResponseMetadata': {'RequestId': 'aeac3058-e422-4f29-bd97-40e651b4b90b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 11 Jun 2024 11:44:59 GMT', 'content-type': 'application/json', 'content-length': '522', 'connection': 'keep-alive', 'x-amzn-requestid': 'aeac3058-e422-4f29-bd97-40e651b4b90b'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': "Okay, let's break this down step-by-step:\n* You originally had 4 cats\n* You just adopted 2 more cats\n* To get the new total, we take the original number and add the number of new cats adopted:"}, {'toolUse': {'toolUseId': 'tooluse_g8l_dIXGS7K13AcX9PJ0cw', 'name': 'math_arithmetic', 'input': {'operation': 'add', 'operand1': '4', 'operand2': '2'}}}]}}, 'stopReason': 'tool_use', 'usage': {'inputTokens': 265, 'outputTokens': 147, 'totalTokens': 412}, 'metrics': {'latencyMs': 7651}}


Claude wants to call the calculator tool:

> ToolUseBlock(id='toolu_staging_01RFker5oMQoY6jErz5prmZg', input={'num1': 4, 'num2': 2}, name='calculator', type='tool_use')

Clearly, this current implementation is not doing what we want (mostly because we set it up to fail). 

Next, let's force Claude to **always** use the `print_sentiment_scores` tool by updating `tool_choice`:

```py
tool_choice={"type": "tool", "name": "print_sentiment_scores"}
```

In addition to setting `type` to `tool`, we must provide a particular tool name.

In [15]:
def analyze_tweet_sentiment(query):
    converse_response = bedrock_client.converse(
        modelId=MODEL_ID,
        inferenceConfig={"maxTokens": 4096},
        messages=[{
             "role": "user",
                "content": [{"text": query}]
        }],
        toolConfig = {
                'tools': tools,
                "toolChoice": {
                    'tool': {'name': 'print_sentiment_scores'}
                },
        }
    )
    print(converse_response)

Now if we try prompting Claude with the same prompts from earlier, it's always going to call the `print_sentiment_scores` tool:

In [16]:
analyze_tweet_sentiment("Holy cow, I just made the most incredible meal!")

{'ResponseMetadata': {'RequestId': '00c46900-f8f3-4e55-95ec-9409937a413e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 11 Jun 2024 11:45:13 GMT', 'content-type': 'application/json', 'content-length': '335', 'connection': 'keep-alive', 'x-amzn-requestid': '00c46900-f8f3-4e55-95ec-9409937a413e'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'toolUse': {'toolUseId': 'tooluse_iCfAEUoqREu18__8Uqz_6Q', 'name': 'print_sentiment_scores', 'input': {'positive_score': 0.9, 'negative_score': 0.1, 'neutral_score': 0.0}}}]}}, 'stopReason': 'tool_use', 'usage': {'inputTokens': 527, 'outputTokens': 79, 'totalTokens': 606}, 'metrics': {'latencyMs': 1115}}


Claude calls our `print_sentiment_scores` tool:

> ToolUseBlock(id='toolu_staging_01FMRQ9pZniZqFUGQwTcFU4N', input={'positive_score': 0.9, 'negative_score': 0.0, 'neutral_score': 0.1}, name='print_sentiment_scores', type='tool_use')

Even if we try to trip up Claude with a "Math-y" tweet, it still always calls the `print_sentiment_scores` tool:

In [17]:
analyze_tweet_sentiment("I love my cats! I had four and just adopted 2 more! Guess how many I have now?")

{'ResponseMetadata': {'RequestId': '8769b55d-cee1-4fb8-a253-743330c0867f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 11 Jun 2024 11:45:29 GMT', 'content-type': 'application/json', 'content-length': '335', 'connection': 'keep-alive', 'x-amzn-requestid': '8769b55d-cee1-4fb8-a253-743330c0867f'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'toolUse': {'toolUseId': 'tooluse_kgl6IMeeTsG6ofCpmYpiHQ', 'name': 'print_sentiment_scores', 'input': {'positive_score': 0.9, 'negative_score': 0.0, 'neutral_score': 0.1}}}]}}, 'stopReason': 'tool_use', 'usage': {'inputTokens': 540, 'outputTokens': 79, 'totalTokens': 619}, 'metrics': {'latencyMs': 1643}}


Even though we're forcing Claude to call our `print_sentiment_scores` tool, we should still employ some basic prompt engineering:

In [18]:
def analyze_tweet_sentiment(query):

    prompt = f"""
    Analyze the sentiment in the following tweet: 
    <tweet>{query}</tweet>
    """
    
    converse_response = bedrock_client.converse(
            modelId=MODEL_ID,
            inferenceConfig={"maxTokens": 4096},
            messages=[{
                "role": "user",
                "content": [{"text": prompt}]
            }],
            toolConfig = {
                'tools': tools,
                "toolChoice": { 'auto': {} },
            }
    )
    print(converse_response)

## Any

The final option for `tool_choice` is `any` which allows us to tell Claude "you must call a tool, but you can pick which one".  Imagine we want to create a SMS chatbot using Claude.  The only way for this chatbot to actually "communicate" with a user is via SMS text message. 

In the example below, we make a very simple text-messaging assistant that has access to two tools:
* `send_text_to_user` sends a text message to a user
* `get_customer_info` looks up customer data based on a username

The idea is to create a chatbot that always calls one of these tools and never responds with a non-tool response.  In all situations, Claude should either respond back by trying to send a text message or calling `get_customer_info` to get more customer information.

Most importantly, we set `tool_choice` to "any":

```py
tool_choice={"type": "any"}
```

In [19]:
def send_text_to_user(text):
    # Sends a text to the user
    # We'll just print out the text to keep things simple:
    print(f"TEXT MESSAGE SENT: {text}")

def get_customer_info(username):
    return {
        "username": username,
        "email": f"{username}@email.com",
        "purchases": [
            {"id": 1, "product": "computer mouse"},
            {"id": 2, "product": "screen protector"},
            {"id": 3, "product": "usb charging cable"},
        ]
    }


tools = [
    {
        "toolSpec": {
            "name": "send_text_to_user",
            "description": "Sends a text message to a user",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "text": {
                            "type": "string",
                            "description": "The piece of text to be sent to the user via text message"
                        }
                    },
                    "required": ["text"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "get_customer_info",
            "description": "gets information on a customer based on the customer's username.  Response includes email, username, and previous purchases. Only call this tool once a user has provided you with their username",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "username": {
                            "type": "string",
                            "description": "The username of the user in question. "
                        }
                    },
                    "required": ["username"]
                }
            }
        }
    }
]

system_prompt = """
All your communication with a user is done via text message.
Only call tools when you have enough information to accurately call them.  
Do not call the get_customer_info tool until a user has provided you with their username. This is important.
If you do not know a user's username, simply ask a user for their username.
"""

def sms_chatbot(user_message):
    # messages = [{"role": "user", "content":user_message}]
    messages = [{
        "role": "user",
        "content": [{"text": user_message}]
    }]

    converse_response = bedrock_client.converse(
            modelId=MODEL_ID,
            inferenceConfig={"maxTokens": 4096},
            system=[{"text" : system_prompt}],
            messages=messages,
            toolConfig = {
                'tools': tools,
                "toolChoice": { 'any': {} },
            }
    )

    print(f"stopReason={converse_response['stopReason']}")
    if converse_response['stopReason'] == "tool_use":
        last_content_block = converse_response['output']['message']['content'][-1]
        if last_content_block.get("toolUse"):
            tool_name = last_content_block.get("toolUse").get('name')
            tool_inputs = last_content_block.get("toolUse").get('input')
            print(f"=======Claude Wants To Call The {tool_name} Tool=======")
            if tool_name == "send_text_to_user":
                send_text_to_user(tool_inputs["text"])
            elif tool_name == "get_customer_info":
                print(get_customer_info(tool_inputs["username"]))
            else:
                print("Oh dear, that tool doesn't exist!")
            
    else:
        print("No tool was called. This shouldn't happen!")


Let's start simple:

In [20]:
sms_chatbot("Hey there! How are you?")

stopReason=tool_use
=======Claude Wants To Call The send_text_to_user Tool=======
TEXT MESSAGE SENT: Hello! I'm an AI assistant. How can I help you today?


Claude responds back by calling the `send_text_to_user` tool.

Next, we'll ask Claude something a bit trickier:

In [21]:
sms_chatbot("I need help looking up an order")

stopReason=tool_use
=======Claude Wants To Call The send_text_to_user Tool=======
TEXT MESSAGE SENT: Hello, I'd be happy to look up information on your order. Could you please provide me with your username? I'll need that to access your account details and order history.


Claude wants to send a text message, asking a user to provide their username.

Now, let's see what happens when we provide Claude with our username:

In [22]:
sms_chatbot("I need help looking up an order.  My username is jenny76")

stopReason=tool_use
=======Claude Wants To Call The get_customer_info Tool=======
{'username': 'jenny76', 'email': 'jenny76@email.com', 'purchases': [{'id': 1, 'product': 'computer mouse'}, {'id': 2, 'product': 'screen protector'}, {'id': 3, 'product': 'usb charging cable'}]}


Claude calls the `get_customer_info` tool, just as we hoped! 

Even if we send Claude a gibberish message, it will still call one of our tools:

In [23]:
sms_chatbot("askdj aksjdh asjkdbhas kjdhas 1+1 ajsdh")

stopReason=tool_use
=======Claude Wants To Call The send_text_to_user Tool=======
TEXT MESSAGE SENT: I'm sorry, I didn't understand your request. Could you please rephrase what you need help with? I'll do my best to assist you.
